<a href="https://colab.research.google.com/github/caroheymes/jedha_final_project/blob/master/page_rank_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Collecte des page rank**

Le site https://www.calcul-pagerank.fr/ semble donner un calcul crédible du page_rank Google. 

Ce score indique la qualité du maillage interne, c'est à dire une évaluation de la qualité des liens qui entrent et qui sortent sur une page donnée. Le Google page rank est confidentiel depuis 2016. 

En soi, le calcul de l'approximation du page rank devrait faire l'objet d'un projet spécifique. La litterature sur le sujet est assez abondante et devrait permettre d'obtenir de bien meilleurs résultats. Calculer un page rank nécessiterait de crawler l'ensemble d'un site. Ce calcul nécessite en effet de prendre en compte l'ensemble des liens entrants et sortants sur une page déterminée.

Pour aller à l'essentiel et pour gagner du temps (une approximation suffit), nous avons donc utilisé le site calcul-page_rank comme source.

**Méthode:**

Le score est indiqué dans une image obtenue à partir d'une url du type :
`https://www.calcul-pagerank.fr/client/www.calcul-pagerank.fr.gif`

 ✅ Ici pas besoin de machine learning : il n'y a que 10 images pour 10 scores de 0 à 9 !

 6.gif


L'image ci-dessus représente une note de 6.

Cette image peut être **analysée comme un tableau numpy**. 


Il est très facile de comparer des tableaux entre eux.
Il "suffit" donc de :
*   collecter chaque image, pour chaque site (...)
*   stocker le tableau numpy correspondant 
*   rapprocher ce tableau d'un des 10 tableaux déjà connux.




# Capture des données de page rank

In [ ]:
# Chargement des packages
import requests
import PIL.Image
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt
import time

In [ ]:
# Création de la fonction de génération du tableau numpy de score pour un site donné

def generate_page_rank_array(dns):
  #Construction de la requête
  if dns[:2] == 'fr':        
    elem = 'www' + dns[2:]
  else:
    elem = dns
  url = 'https://www.calcul-pagerank.fr/client/' + elem + '.gif'
  s = requests.session()
  img_data = s.get(url).content
  img_name = dns + '.gif'
  
  #Je génère l'image
  with open(img_name, 'wb') as handler:
      handler.write(img_data)
  #Je génère l'array pour cet enregistrement
  image = PIL.Image.open(img_name)
  #image array name
  name = 'page_rank_' + dns
  name = name
 
  # J'affecte à name le contenu de l'array
  name = np.array(image)
  
  # save to csv file
  csv_name = dns + '.csv'
  savetxt(csv_name, name, delimiter=',')

  return name


# Collecte des images et de tableaux 

all_dns = pd.read_csv('/content/drive/MyDrive/dns.csv').drop(columns=['Unnamed: 0'])
mylist = all_dns.displayLink.values
page_rank = pd.DataFrame()

for e in mylist:
  try:
    generate_page_rank_array(e)
    elem = [e, generate_page_rank_array(e)]
  except:
    elem = [e, '']
  temp = pd.DataFrame(elem).T
  page_rank = pd.concat([page_rank, temp], axis=0)
page_rank.to_csv('+++++dns_avec tableau.csv')
page_rank

# Fusion des tables de données

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**On dénombre le nombre de gifs générés à l'issue de la collecte**

In [ ]:
os.chdir('/content/drive/MyDrive/test')

In [ ]:
import os
import pandas as pd
from fnmatch import fnmatch
root = '/content/drive/MyDrive/test'
pattern = "*.gif"
i=0
liste = []
for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            liste.append(name)
            i+=1

In [ ]:
#On rapproche les images générées des nom de domaines
done = pd.DataFrame(liste)
done = done.rename(columns={0 : 'img_file'})
done['dns'] = [c[:c.rfind('.')] for c in done.img_file]
done.head()

,img_file,dns
0,www.carriere-info.fr.gif,www.carriere-info.fr
1,www.emploi.vendee.fr.gif,www.emploi.vendee.fr
2,www.exaequo-interim.fr.gif,www.exaequo-interim.fr
3,www.limoges-emplois.com.gif,www.limoges-emplois.com
4,www.sauvaje.fr.gif,www.sauvaje.fr


**Merge avec les dns de base**

In [ ]:
base = pd.read_csv('/content/drive/MyDrive/ZZZTEST/dns.csv')
result = base.merge(done, left_on='displayLink', right_on ='dns', how='left')

In [ ]:
result.head()

,Unnamed: 0,displayLink,img_file,dns
0,0,candidat.pole-emploi.fr,candidat.pole-emploi.fr.gif,candidat.pole-emploi.fr
1,1,fr.indeed.com,fr.indeed.com.gif,fr.indeed.com
2,2,www.estjob.com,www.estjob.com.gif,www.estjob.com
3,3,fr.indeed.com,fr.indeed.com.gif,fr.indeed.com
4,4,candidat.pole-emploi.fr,candidat.pole-emploi.fr.gif,candidat.pole-emploi.fr


**Concatenation de l'ensemble des arrays numpy correspondant aux scores**

In [ ]:
# liste de tous les fichiers d'un dossier
# import os
from fnmatch import fnmatch
root = '/content/drive/MyDrive/test'
pattern = "*rank*"
i=0
liste = []
for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            liste.append(name)
            i+=1
liste

['+++++bloc2_rank.csv',
 '+++++page_rank.csv',
 '+++++bloc1_rank.csv',
 '+++++bloc3_rank.csv']

In [ ]:
total = pd.DataFrame()
for e in liste:
  temp = pd.read_csv(e)
  total = pd.concat([total, temp], axis=0)
total.drop(columns=['Unnamed: 0'], inplace=True)
total = total.rename(dict(zip(total.columns, ['col1', 'col2'])))
total.rename(columns={0:'col0'}, inplace=True)

In [ ]:
total.sample(5)

,0,1
238,www.manpower.fr,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...
189,www.nordjob.com,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...
903,www.jobijoba.com,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...
4850,www.leboncoin.fr,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...
2000,fr.indeed.com,[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14...


**Mise en place d'une table de correspondance tableau numpy versus page_rank score**

In [ ]:
matching = total.drop_duplicates(subset=['1'])
matching

,0,1
0,candidat.pole-emploi.fr,[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14...
1,fr.indeed.com,[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14...
2,www.estjob.com,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...
5,fr.linkedin.com,[[13 13 13 13 13 13 13 13 13 13 13 13 13 13 13...
13,www.crit-job.com,[[10 10 10 10 10 10 10 10 10 10 10 10 10 10 10...
14,www.glassdoor.fr,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...
26,www.lejobadequat.com,NaN
48,www.ouest-france.fr,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...
142,www.pmejob.fr,[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14...
268,www.eterritoire.fr,[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14...


In [ ]:
matching['0'].values


array(['candidat.pole-emploi.fr', 'fr.indeed.com', 'www.estjob.com',
       'fr.linkedin.com', 'www.crit-job.com', 'www.glassdoor.fr',
       'www.lejobadequat.com', 'www.ouest-france.fr', 'www.pmejob.fr',
       'www.eterritoire.fr'], dtype=object)

In [ ]:
# Création de la table de correspondance
corresp = pd.DataFrame.from_dict({
    'candidat.pole-emploi.fr' :6,
    'fr.indeed.com'           :8,
    'www.estjob.com'          :5,
    'fr.linkedin.com'         :9,
    'www.crit-job.com'        :4,
    'www.glassdoor.fr'        :1,
    'www.lejobadequat.com'    :0,
    'www.ouest-france.fr'     :7,
    'www.pmejob.fr'           :3,
    'www.eterritoire.fr'      :2
                                }, orient='index').reset_index()

page_rank = matching.merge(corresp, how='left', left_on='0', right_on='index').drop(columns=['0', 'index'])
page_rank = page_rank.rename(columns={'1' : 'np_array', 0: 'score'})
page_rank

,np_array,score
0,[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14...,6
1,[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14...,8
2,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...,5
3,[[13 13 13 13 13 13 13 13 13 13 13 13 13 13 13...,9
4,[[10 10 10 10 10 10 10 10 10 10 10 10 10 10 10...,4
5,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...,1
6,NaN,0
7,[[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12...,7
8,[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14...,3
9,[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14...,2


**Merge des dns et des tableaux numpy de score**

In [ ]:

page_rank = total.merge(page_rank, left_on='1', right_on='np_array', how='left')
page_rank = page_rank[['0', 'score']]
page_rank.rename(columns={'0' : 'dns'}, inplace=True)


In [ ]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/11_r9Tp2C6HqDG5QsBObNiqO23rMSOx-G/test'

In [ ]:
os.chdir("/content/drive/MyDrive/111_goldfinger/data_collection")

In [ ]:
# !mkdir results

In [ ]:
page_rank.to_csv('./results/page_rank_done.csv')
page_rank.sample(10)

,dns,score
5014,www.startpeople.fr,1
273,www.ouestjob.com,6
5250,candidat.pole-emploi.fr,6
883,fr.jooble.org,6
4566,www.meteojob.com,5
2011,www.meteojob.com,5
3465,www.regionsjob.com,6
1735,www.jobijoba.com,5
4299,www.meteojob.com,5
6198,www.manpower.fr,1


**Vérification de l'export des données**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
os.chdir('./results')

In [ ]:
!ls

page_rank_done.csv
